### Airline Data Analysis Using SparkML

In [0]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

#### 1. Load Airline Delay Cancellation data

In [0]:
# Store CSV files and read
path = "/FileStore/tables/2009.csv"

# Load the files into separate DataFrames
df = spark.read.format("csv").option("header", "true").load(path)
df.printSchema()

# Extend the DF wiht data from other years
for year in ["2010", "2011", "2012", "2013", "2014", "2015"]:
    path = f"/FileStore/tables/{year}.csv"
    temp_df = spark.read.format("csv").option("header", "true").load(path)
    df = df.union(temp_df)

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: strin

In [0]:
display(df.limit(25))

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,null,0.0,62.0,68.0,42.0,199.0,null,null,null,null,null,null
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,null,0.0,82.0,75.0,43.0,213.0,null,null,null,null,null,null
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,null,0.0,70.0,62.0,36.0,199.0,null,null,null,null,null,null
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,null,0.0,77.0,56.0,37.0,199.0,null,null,null,null,null,null
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,null,0.0,105.0,77.0,40.0,213.0,null,null,null,null,null,null
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,null,0.0,147.0,130.0,96.0,745.0,null,null,null,null,null,null
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,null,0.0,117.0,108.0,91.0,554.0,null,null,null,null,null,null
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,null,0.0,80.0,79.0,36.0,199.0,null,null,null,null,null,null
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,null,0.0,83.0,73.0,41.0,199.0,null,null,null,null,null,null
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,null,0.0,68.0,61.0,39.0,199.0,null,null,null,null,null,null


#### 2. Data Preprocessing

In [0]:
# Drop unnamed columns
df = df.drop("Unnamed: 27") 

#Convert columns to integer type
cols_to_int = ["CRS_DEP_TIME", "DEP_TIME", "DEP_DELAY", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "CRS_ARR_TIME", "ARR_TIME", "ARR_DELAY", "CANCELLED", "CRS_ELAPSED_TIME", "ACTUAL_ELAPSED_TIME", "AIR_TIME", "DISTANCE", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
for col in cols_to_int:
    df = df.withColumn(col, df[col].cast(IntegerType()))

df_cancelled = df.filter(F.col("CANCELLED")>0)

#Find columns that are null if the flight is cancelled
null_counts = df_cancelled.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
df_cancelled_size = df_cancelled.count()
for col in null_counts:
    null_counts[col] = (null_counts[col], round(null_counts[col]/df_cancelled_size*100))
print(null_counts)

{'FL_DATE': (0, 0), 'OP_CARRIER': (0, 0), 'OP_CARRIER_FL_NUM': (0, 0), 'ORIGIN': (0, 0), 'DEST': (0, 0), 'CRS_DEP_TIME': (0, 0), 'DEP_TIME': (679642, 96), 'DEP_DELAY': (679642, 96), 'TAXI_OUT': (700508, 99), 'WHEELS_OFF': (700508, 99), 'WHEELS_ON': (708071, 100), 'TAXI_IN': (708071, 100), 'CRS_ARR_TIME': (0, 0), 'ARR_TIME': (708071, 100), 'ARR_DELAY': (708071, 100), 'CANCELLED': (0, 0), 'CANCELLATION_CODE': (0, 0), 'DIVERTED': (0, 0), 'CRS_ELAPSED_TIME': (21, 0), 'ACTUAL_ELAPSED_TIME': (708071, 100), 'AIR_TIME': (708071, 100), 'DISTANCE': (0, 0), 'CARRIER_DELAY': (708071, 100), 'WEATHER_DELAY': (708071, 100), 'NAS_DELAY': (708071, 100), 'SECURITY_DELAY': (708071, 100), 'LATE_AIRCRAFT_DELAY': (708071, 100)}


In [0]:
#drop colums that are mostly null for cancelled flights
cols_to_drop = [k for k, v in null_counts.items() if v[1] > 90]
df = df.drop(*cols_to_drop)

# Remove records containing null values (exept cancellation code)
cols = df.columns
cols.remove("CANCELLATION_CODE")
df = df.na.drop(subset=cols)

display(df.limit(25))

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,DISTANCE
2009-01-01,XE,1204,DCA,EWR,1100,1202,0,null,0.0,62,199
2009-01-01,XE,1206,EWR,IAD,1510,1632,0,null,0.0,82,213
2009-01-01,XE,1207,EWR,DCA,1100,1210,0,null,0.0,70,199
2009-01-01,XE,1208,DCA,EWR,1240,1357,0,null,0.0,77,199
2009-01-01,XE,1209,IAD,EWR,1715,1900,0,null,0.0,105,213
2009-01-01,XE,1212,ATL,EWR,1915,2142,0,null,0.0,147,745
2009-01-01,XE,1212,CLE,ATL,1645,1842,0,null,0.0,117,554
2009-01-01,XE,1214,DCA,EWR,1915,2035,0,null,0.0,80,199
2009-01-01,XE,1215,EWR,DCA,1715,1838,0,null,0.0,83,199
2009-01-01,XE,1217,EWR,DCA,1300,1408,0,null,0.0,68,199


#### 3. Data Analysis

In [0]:
# Top 10 airlines with the most flight operations
display(df.groupBy("OP_CARRIER").count().sort(F.col("count").desc()).limit(10))

OP_CARRIER,count
WN,8115099
DL,5046291
OO,4176455
AA,3954500
EV,3693024
UA,3075545
MQ,2920259
US,2657286
B6,1597829
FL,1218388


In [0]:
#Visualizing the proportion of the total flight cancellation codes
df.na.drop(subset="CANCELLATION_CODE").groupBy("CANCELLATION_CODE").count()\
.toPandas().set_index("CANCELLATION_CODE").plot.pie(y='count')  

In [0]:
display(df.groupBy("CANCELLED").count())

CANCELLED,count
1,708050
0,42343151


#### 4. Model Prediction

In [0]:
#CANCELLATION_CODE should not be used to predict cancelled flights
df = df.drop("CANCELLATION_CODE")

In [0]:
#Use StringIndexer to convert string labels to label indices
date_indexer = StringIndexer(inputCol="FL_DATE", outputCol="FL_DATE_IDX")
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_IDX")
flnum_indexer = StringIndexer(inputCol="OP_CARRIER_FL_NUM", outputCol="OP_CARRIER_FL_NUM_IDX")
origin_indexer = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_IDX")
destination_indexer = StringIndexer(inputCol="DEST", outputCol="DEST_IDX")
df = date_indexer.fit(df).transform(df)
df = carrier_indexer.fit(df).transform(df)
df = flnum_indexer.fit(df).transform(df)
df = origin_indexer.fit(df).transform(df)
df = destination_indexer.fit(df).transform(df)
df = df.drop("FL_DATE", "OP_CARRIER", "OP_CARRIER_FL_NUM", "ORIGIN", "DEST", "DIVERTED")
display(df.limit(25))

CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,FL_DATE_IDX,OP_CARRIER_IDX,OP_CARRIER_FL_NUM_IDX,ORIGIN_IDX,DEST_IDX
1100,1202,0,62,199,1832.0,10.0,1382.0,23.0,14.0
1510,1632,0,82,213,1832.0,10.0,1203.0,14.0,27.0
1100,1210,0,70,199,1832.0,10.0,1255.0,14.0,23.0
1240,1357,0,77,199,1832.0,10.0,917.0,23.0,14.0
1715,1900,0,105,213,1832.0,10.0,837.0,27.0,14.0
1915,2142,0,147,745,1832.0,10.0,620.0,0.0,14.0
1645,1842,0,117,554,1832.0,10.0,620.0,35.0,0.0
1915,2035,0,80,199,1832.0,10.0,908.0,23.0,14.0
1715,1838,0,83,199,1832.0,10.0,1013.0,14.0,23.0
1300,1408,0,68,199,1832.0,10.0,1197.0,14.0,23.0


In [0]:
#Balance the dataset
major_df = df.filter(F.col("CANCELLED") == 0)
minor_df = df.filter(F.col("CANCELLED") == 1)
ratio = int(major_df.count()/minor_df.count())

sampled_majority_df = major_df.sample(False, 1/ratio, seed=1)
balanced_df = sampled_majority_df.unionAll(minor_df)
display(balanced_df.limit(25))

CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,FL_DATE_IDX,OP_CARRIER_IDX,OP_CARRIER_FL_NUM_IDX,ORIGIN_IDX,DEST_IDX
1546,1724,0,98,418,1832.0,10.0,1950.0,35.0,18.0
1759,1951,0,112,418,1832.0,10.0,2383.0,18.0,35.0
1848,2008,0,80,314,1832.0,10.0,2314.0,19.0,35.0
1858,2007,0,69,192,1832.0,10.0,1880.0,58.0,35.0
1155,1452,0,177,1080,1832.0,10.0,2053.0,24.0,35.0
2135,2313,0,98,278,1832.0,10.0,2347.0,14.0,68.0
1345,1557,0,132,668,1832.0,10.0,2000.0,28.0,6.0
853,1230,0,157,1008,1832.0,10.0,2298.0,11.0,14.0
600,804,0,124,595,1832.0,10.0,2508.0,100.0,6.0
1140,1324,0,164,936,1832.0,10.0,2322.0,6.0,62.0


In [0]:
print(balanced_df.filter(F.col("CANCELLED") == 0).count())
print(balanced_df.filter(F.col("CANCELLED") == 1).count())

718604
708050


In [0]:
#Vector Assembler
from pyspark.ml.feature import VectorAssembler
assembler_inputs = [col for col in balanced_df.columns if col != 'CANCELLED']
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

balanced_df = vec_assembler.transform(balanced_df).drop(*assembler_inputs)

In [0]:
#Split the data into a 70/30 test and train ratio
train, test = balanced_df.randomSplit([0.7, 0.3], seed=1)
#print(train.count(), train.filter(F.col("CANCELLED") == 0).count(), train.filter(F.col("CANCELLED") == 1).count())
#print(test.count(), test.filter(F.col("CANCELLED") == 0).count(), test.filter(F.col("CANCELLED") == 1).count())

In [0]:
display(train.limit(25))

CANCELLED,features
0,"Map(vectorType -> dense, length -> 9, values -> List(5.0, 626.0, 261.0, 1989.0, 667.0, 1.0, 9.0, 13.0, 16.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(5.0, 626.0, 261.0, 1989.0, 2272.0, 1.0, 9.0, 13.0, 16.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(20.0, 559.0, 219.0, 1536.0, 667.0, 18.0, 348.0, 4.0, 11.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(20.0, 600.0, 220.0, 1536.0, 363.0, 18.0, 348.0, 4.0, 11.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(35.0, 603.0, 208.0, 1635.0, 363.0, 14.0, 1386.0, 7.0, 6.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(35.0, 608.0, 213.0, 1635.0, 680.0, 14.0, 1386.0, 7.0, 6.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(35.0, 742.0, 247.0, 1946.0, 672.0, 1.0, 1714.0, 4.0, 0.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(35.0, 742.0, 247.0, 1946.0, 682.0, 1.0, 1714.0, 4.0, 0.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(35.0, 742.0, 247.0, 1946.0, 2296.0, 1.0, 1714.0, 4.0, 0.0))"
0,"Map(vectorType -> dense, length -> 9, values -> List(50.0, 550.0, 180.0, 1300.0, 665.0, 18.0, 597.0, 8.0, 11.0))"


### Models

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier

dt = DecisionTreeClassifier(labelCol='CANCELLED', 
                            featuresCol='features', 
                            #maxDepth=10,
                            maxBins=7592 
                           )

rf = RandomForestClassifier(labelCol='CANCELLED', 
                            featuresCol='features', 
                            #maxDepth=10,
                            maxBins=7592
                           )

lr = LogisticRegression(labelCol='CANCELLED',
                        featuresCol='features'
                       )

gbt = GBTClassifier(labelCol='CANCELLED', 
                    featuresCol='features', 
                    #maxDepth=10,
                    maxBins=7592
                   )

In [0]:
#Train models
dtModel = dt.fit(train)

In [0]:
rfModel = rf.fit(train)

In [0]:
lrModel = lr.fit(train)

In [0]:
gbtModel = gbt.fit(train)

In [0]:
# Create predictions
dtPredictions = dtModel.transform(test)
rfPredictions = rfModel.transform(test)
lrPredictions = lrModel.transform(test)
gbtPredictions = gbtModel.transform(test)

In [0]:
# Create confusion matrices and output accuracies.
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

dtPred_pd = dtPredictions.select('CANCELLED','prediction').toPandas()
dt_cm = confusion_matrix(list(dtPred_pd['CANCELLED']), list(dtPred_pd['prediction']))
print("Decision tree matrix:")
print(dt_cm)
print(f"Accuracy: {(dt_cm[0][0]+dt_cm[1][1])/(sum(sum(dt_cm)))} %")
print()

rfPred_pd = rfPredictions.select('CANCELLED','prediction').toPandas()
rf_cm = confusion_matrix(list(rfPred_pd['CANCELLED']), list(rfPred_pd['prediction']))
print("Decision tree matrix:")
print(rf_cm)
print(f"Accuracy: {(rf_cm[0][0]+rf_cm[1][1])/(sum(sum(rf_cm)))} %")
print()

lrPred_pd = lrPredictions.select('CANCELLED','prediction').toPandas()
lr_cm = confusion_matrix(list(lrPred_pd['CANCELLED']), list(lrPred_pd['prediction']))
print("Decision tree matrix:")
print(lr_cm)
print(f"Accuracy: {(lr_cm[0][0]+lr_cm[1][1])/(sum(sum(lr_cm)))} %")
print()

gbtPred_pd = gbtPredictions.select('CANCELLED','prediction').toPandas()
gbt_cm = confusion_matrix(list(gbtPred_pd['CANCELLED']), list(gbtPred_pd['prediction']))
print("Decision tree matrix:")
print(gbt_cm)
print(f"Accuracy: {(gbt_cm[0][0]+gbt_cm[1][1])/(sum(sum(gbt_cm)))} %")
print()

Decision tree matrix:
[[172640  42893]
 [ 77630 135152]]
Accuracy: 0.7186113024292868 %

Decision tree matrix:
[[171159  44374]
 [ 69666 143116]]
Accuracy: 0.7337473588363704 %

Decision tree matrix:
[[132563  82970]
 [ 84259 128523]]
Accuracy: 0.6095653899583251 %

Decision tree matrix:
[[167892  47641]
 [ 57213 155569]]
Accuracy: 0.7551941911910627 %



Actual order above:
1. Decision tree
2. Random Forest
3. Logistic regression
4. Gradient Boosted Trees